In [ ]:
!pip install -Uq optuna
!pip install -Uq xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00


In [ ]:
import numpy as np  # для работы с массивами и математическими функциями
import pandas as pd  # для обработки и анализа данных, особенно табличных данных
import os
import glob
import optuna  # для оптимизации гиперпараметров модели с помощью продвинутых методов поиска
from sklearn.model_selection import train_test_split  # для разделения данных на обучающую и тестовую выборки
from sklearn.metrics import roc_auc_score
import xgboost as xgb


In [ ]:
df_hack_test = pd.read_csv('Test_датасет_альфа')
df_hack_test.head()

,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_409,feature_410,feature_411,feature_412,feature_413,feature_414,feature_415,feature_416,feature_417,feature_418
0,test,38731,0.262983,-1.153775,1.0,-0.648932,-0.847237,-0.462393,-1.215373,0.605520,...,-0.236218,-1.058259,0.0,0.835147,-0.176562,-1.455333,-0.766280,-1.365915,0.454763,0.809724
1,test,38732,-0.698407,-1.529096,1.0,0.033011,-1.166594,-0.889886,-0.900391,-1.201925,...,-1.157688,0.334938,0.0,-0.231860,-0.080340,-0.493846,-1.245681,-1.632521,0.246708,-0.226893
2,test,38733,-0.331403,-0.538437,1.0,-1.531462,-0.924031,-1.062342,-1.789274,-1.097902,...,-1.159555,0.351526,0.0,-1.375061,-0.156735,-0.437773,-1.851818,0.984694,0.630962,0.279446
3,test,38734,0.675162,-0.077064,1.0,-1.893065,0.610631,-1.404934,-2.650392,-1.752463,...,0.180918,0.269593,0.0,1.054666,-0.219652,-0.438285,-0.707442,-1.498935,0.962498,-0.390251
4,test,38735,-2.162385,-1.737901,1.0,-1.899384,-1.384151,0.106314,-0.814678,1.111808,...,0.696544,1.006378,0.0,1.107787,-0.282357,-0.235000,-1.794316,-0.504509,-0.385182,-1.130777


In [ ]:
df_hack_train = pd.read_csv('Train_датасет_альфа')
df_hack_train.head()

,target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_409,feature_410,feature_411,feature_412,feature_413,feature_414,feature_415,feature_416,feature_417,feature_418
0,0,train,0,0.468142,-1.045346,0.0,0.384487,0.435121,-1.178548,0.124543,...,-0.361507,-1.026853,0.0,1.418600,-0.929668,1.284014,0.731842,0.801786,-0.728297,-0.412095
1,0,train,1,-0.760983,0.515132,0.0,-1.673905,-0.393862,-1.584207,-0.439778,...,-0.546275,-1.489542,0.0,-0.622007,-0.473156,0.780020,0.648577,0.646100,-0.789362,0.083349
2,0,train,2,1.658855,0.915052,0.0,-0.581082,0.477199,-0.622226,0.390642,...,-0.485999,0.586012,0.0,0.361481,-0.364566,-1.318596,-0.385155,0.140133,0.123245,-0.670030
3,0,train,3,-0.638854,0.314099,0.0,0.000919,1.102342,-0.807371,0.329158,...,0.321985,-0.075827,0.0,-1.629672,0.876864,0.411271,0.433440,0.997364,2.829590,-1.275588
4,0,train,4,-1.091376,0.859811,0.0,-0.505439,1.665086,-0.912464,-0.332054,...,0.828886,0.140387,0.0,-0.624304,-2.197691,-1.479267,-0.465917,-0.014757,-0.320434,-0.511896


In [ ]:
X = df_hack_train.drop(['target','smpl','id'], axis=1)
y = df_hack_train['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def objective(trial):
    # Гиперпараметры для XGBoos
    param = {
        'tree_method': 'hist',  # Используем метод построения дерева
        'device': 'cuda',
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "max_depth": trial.suggest_int("xgb_max_depth", 3, 20),
        "learning_rate": trial.suggest_loguniform("xgb_learning_rate", 1e-6, 1e-1),
        "subsample": trial.suggest_uniform("xgb_subsample", 0.3, 1.0),
        "colsample_bytree": trial.suggest_uniform("xgb_colsample_bytree", 0.3, 1.0),
        "n_estimators": trial.suggest_int("xgb_n_estimators", 100, 1500),
    }

    # Обучение модели
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    model = xgb.train(param, dtrain, evals=[(dval, 'validation')],num_boost_round=2000, early_stopping_rounds=300, verbose_eval=False)

    # Предсказания и расчёт AUC
    preds = model.predict(dval)
    auc = roc_auc_score(y_val, preds)

    return auc

In [ ]:
import time

# Сохранение времени начала для первой итерации
start_time = time.time()

def print_progress(study, trial):
    global start_time
    # Проверка, является ли номер испытания кратным 5
    if trial.number % 10 == 0:
        # Вычисление времени, затраченного с момента предыдущей 5-й итерации (или начала)
        elapsed_time = time.time() - start_time
        start_time = time.time()  # Обновляем стартовое время для следующей итерации

        print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}")
        print(f"Time elapsed for last 10 trials: {elapsed_time:.2f} seconds")
        print(f"Best trial so far: Trial {study.best_trial.number} with value: {study.best_trial.value}")

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
# Запуск Optuna для оптимизации гиперпараметров всех моделей
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, callbacks=[print_progress],n_jobs=-1)

Trial 0 finished with value: 0.504741641337386 and parameters: {'xgb_max_depth': 6, 'xgb_learning_rate': 2.589000879500497e-06, 'xgb_subsample': 0.6025302656691065, 'xgb_colsample_bytree': 0.5787054224357844, 'xgb_n_estimators': 553}
Time elapsed for last 10 trials: 10.93 seconds
Best trial so far: Trial 0 with value: 0.504741641337386
Trial 10 finished with value: 0.5243039513677812 and parameters: {'xgb_max_depth': 15, 'xgb_learning_rate': 8.2045397091365e-06, 'xgb_subsample': 0.4858655554815457, 'xgb_colsample_bytree': 0.9549440083549792, 'xgb_n_estimators': 397}
Time elapsed for last 10 trials: 58.29 seconds
Best trial so far: Trial 3 with value: 0.7671003039513677
Trial 20 finished with value: 0.555209726443769 and parameters: {'xgb_max_depth': 10, 'xgb_learning_rate': 0.0002393783811745841, 'xgb_subsample': 0.32690336918359364, 'xgb_colsample_bytree': 0.8684530412462742, 'xgb_n_estimators': 868}
Time elapsed for last 10 trials: 84.44 seconds
Best trial so far: Trial 12 with value

In [ ]:
# Обучение финальной модели с оптимизированными параметрами
best_params = study.best_params
final_model = xgb.train(
    {**best_params, "objective": "binary:logistic", "eval_metric": "auc", "tree_method": "hist", "device": "cuda"},
    xgb.DMatrix(X_train, label=y_train),
    num_boost_round=10000,
    evals=[(xgb.DMatrix(X_val, label=y_val), "validation")],
    early_stopping_rounds=100
)

[0]	validation-auc:0.50917
[1]	validation-auc:0.48621
[2]	validation-auc:0.65671
[3]	validation-auc:0.65480
[4]	validation-auc:0.64119
[5]	validation-auc:0.62923
[6]	validation-auc:0.68580
[7]	validation-auc:0.64874
[8]	validation-auc:0.66815
[9]	validation-auc:0.67105


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:39:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "xgb_colsample_bytree", "xgb_learning_rate", "xgb_max_depth", "xgb_n_estimators", "xgb_subsample" } are not used.

  warnings.warn(smsg, UserWarning)


[10]	validation-auc:0.67319
[11]	validation-auc:0.70119
[12]	validation-auc:0.70179
[13]	validation-auc:0.70032
[14]	validation-auc:0.71061
[15]	validation-auc:0.68802
[16]	validation-auc:0.69919
[17]	validation-auc:0.70013
[18]	validation-auc:0.70281
[19]	validation-auc:0.70130
[20]	validation-auc:0.68355
[21]	validation-auc:0.67971
[22]	validation-auc:0.67613
[23]	validation-auc:0.68467
[24]	validation-auc:0.67995
[25]	validation-auc:0.67801
[26]	validation-auc:0.67759
[27]	validation-auc:0.67112
[28]	validation-auc:0.67837
[29]	validation-auc:0.68046
[30]	validation-auc:0.68464
[31]	validation-auc:0.67820
[32]	validation-auc:0.67443
[33]	validation-auc:0.67458
[34]	validation-auc:0.68153
[35]	validation-auc:0.68574
[36]	validation-auc:0.68357
[37]	validation-auc:0.67917
[38]	validation-auc:0.68090
[39]	validation-auc:0.67966
[40]	validation-auc:0.67813
[41]	validation-auc:0.67842
[42]	validation-auc:0.67691
[43]	validation-auc:0.67224
[44]	validation-auc:0.67093
[45]	validation-auc:

In [ ]:
# Оценка финальной модели на тестовой выборке
final_preds = final_model.predict(xgb.DMatrix(X_val))
final_auc = roc_auc_score(y_val, final_preds)
print("AUC на тестовой выборке:", final_auc)

AUC на тестовой выборке: 0.6980668693009119


In [ ]:
# Сохранение модели
final_model.save_model("Test Alpha Hack XGBoost1.json")
print("Модель сохранена как 'Test Alpha Hack XGBoost.json'.")

Модель сохранена как 'Test Alpha Hack XGBoost.json'.


In [ ]:
# Загрузка модели (при необходимости)
loaded_model = xgb.Booster()
loaded_model.load_model("Test Alpha Hack XGBoost.json")
print("Модель успешно загружена.")